# Mod 6 Final Project

Jeremy Owens, Bridget Boakye

### Problem Statement:

We need to find Pulsars, know where to focus telescopes for Pulsar analysis

## Import

In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set_style('darkgrid')
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, KFold
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras import initializers, regularizers
from keras.optimizers import RMSprop, Adam, SGD
from keras.utils.np_utils import to_categorical
# from keras.preprocessing.text import Tokenizer

random_seed = 123

In [8]:
pulsar_df = pd.read_csv('pulsar_stars.csv')
pulsar_df.head(10)

,Mean of the integrated profile,Standard deviation of the integrated profile,Excess kurtosis of the integrated profile,Skewness of the integrated profile,Mean of the DM-SNR curve,Standard deviation of the DM-SNR curve,Excess kurtosis of the DM-SNR curve,Skewness of the DM-SNR curve,target_class
0,140.562500,55.683782,-0.234571,-0.699648,3.199833,19.110426,7.975532,74.242225,0
1,102.507812,58.882430,0.465318,-0.515088,1.677258,14.860146,10.576487,127.393580,0
2,103.015625,39.341649,0.323328,1.051164,3.121237,21.744669,7.735822,63.171909,0
3,136.750000,57.178449,-0.068415,-0.636238,3.642977,20.959280,6.896499,53.593661,0
4,88.726562,40.672225,0.600866,1.123492,1.178930,11.468720,14.269573,252.567306,0
5,93.570312,46.698114,0.531905,0.416721,1.636288,14.545074,10.621748,131.394004,0
6,119.484375,48.765059,0.031460,-0.112168,0.999164,9.279612,19.206230,479.756567,0
7,130.382812,39.844056,-0.158323,0.389540,1.220736,14.378941,13.539456,198.236457,0
8,107.250000,52.627078,0.452688,0.170347,2.331940,14.486853,9.001004,107.972506,0
9,107.257812,39.496488,0.465882,1.162877,4.079431,24.980418,7.397080,57.784738,0


In [9]:
for col in pulsar_df.columns:
    print(min(pulsar_df[col]), max(pulsar_df[col]))

5.8125 192.6171875
24.77204176 98.77891067
-1.876011181 8.069522046
-1.7918859809999999 68.10162173
0.213210702 223.39214049999998
7.370432165 110.64221059999998
-3.1392696110000005 34.53984419
-1.976975603 1191.0008369999998
0 1


In [10]:
pulsar_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17898 entries, 0 to 17897
Data columns (total 9 columns):
 Mean of the integrated profile                  17898 non-null float64
 Standard deviation of the integrated profile    17898 non-null float64
 Excess kurtosis of the integrated profile       17898 non-null float64
 Skewness of the integrated profile              17898 non-null float64
 Mean of the DM-SNR curve                        17898 non-null float64
 Standard deviation of the DM-SNR curve          17898 non-null float64
 Excess kurtosis of the DM-SNR curve             17898 non-null float64
 Skewness of the DM-SNR curve                    17898 non-null float64
target_class                                     17898 non-null int64
dtypes: float64(8), int64(1)
memory usage: 1.2 MB


activation

dropout

hidden layers

neurons in layers

optimizers

batch size

epochs

In [14]:
X = pulsar_df.drop('target_class', axis=1)
y = pulsar_df['target_class']

scalar = StandardScaler()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=random_seed)

scaled_X_train = scalar.fit_transform(X_train)
scaled_X_test = scalar.fit_transform(X_test)
print(scaled_X_train.shape, scaled_X_test.shape)

(16108, 8) (1790, 8)


## Baseline Model